# Based on UniRep tutorial at https://github.com/churchlab/UniRep

Use the 1900-unit model

In [1]:
USE_FULL_1900_DIM_MODEL = True # if True use 1900 dimensional model, else use 64 dimensional one.

## Setup

In [2]:
import tensorflow as tf
import numpy as np

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

## Data formatting and management

Initialize UniRep, also referred to as the "babbler" in our code. You need to provide the batch size you will use and the path to the weight directory.

In [9]:
# Choose which HT-Recruit dataset to use, and set batch size to be the size of that dataset
dataset = "NucRepr"
batch_sizes = {"NucRepr": 3293, "NucAct": 2704, "Tiling": 10538, "Proteome": int(263193./21.)}

batch_size = batch_sizes[dataset] 
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH) # b is the model itself


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


UniRep needs to receive data in the correct format, a (batch_size, max_seq_len) matrix with integer values, where the integers correspond to an amino acid label at that position, and the end of the sequence is padded with 0s until the max sequence length to form a non-ragged rectangular matrix. We provide a formatting function to translate a string of amino acids into a list of integers with the correct codex:

In [3]:
# creating human proteome tiles from sequence
import pandas as pd

genome = np.load("proteome.npy")
genome_data = pd.DataFrame(genome, columns=["id", "sequence"])

genome_tiles = pd.DataFrame(columns=["id", "sequence"])
tile_ids = []; tile_seqs = []
# for each protein in the proteome
for i in range(len(genome_data)):
    # get rid of extraneous * characters
    seq = genome_data.iloc[i]["sequence"].replace("*", "") 
    genome_data.iloc[i]["sequence"] = seq
    # calculate start positions of tiles
    if len(seq) >= 80:
        num_tiles = len(seq)/40.
        if len(seq) != 80:
            num_tiles += 1
        starts = [int(40*s) 
                  for s in range(int(num_tiles)) 
                  if 40*s + 80 <= len(genome_data.iloc[i].sequence)]
        starts.append(len(genome_data.iloc[i].sequence) - 80)
        starts = list(set(starts))
        
        # add tile IDs and sequences to dataframe
        for start in starts:
            end = int(start + 80)
            tile_id = "_".join([genome_data.iloc[i].id, str(start), str(end)])
            tile_seq = genome_data.iloc[i]["sequence"][start:end]
            tile_ids.append(tile_id)
            tile_seqs.append(tile_seq)
            
genome_tiles["id"] = tile_ids
genome_tiles["sequence"] = tile_seqs
print(genome_tiles.head())
seqs_genome = genome_tiles["sequence"]

263193
263193
                          id  \
0     ENST00000367772.8_0_80   
1  ENST00000367772.8_160_240   
2  ENST00000367772.8_280_360   
3  ENST00000367772.8_320_400   
4  ENST00000367772.8_480_560   

                                            sequence  
0  MGSENSALKSYTLREPPFTLPSGLAVYPAVLQDGKFASVFVYKREN...  
1  QSIRDPASIPPEEMSPEFTTLPECHGHARDAFSFGTLVESLLTILN...  
2  LSEELIASRLVPLLLNQLVFAEPVAVKSFLPYLLGPKKDHAQGETP...  
3  AQGETPCLLSPALFQSRVIPVLLQLFEVHEEHVRMVLLSHIEAYVE...  
4  QRDYYNTLLQTGDPFSQPIKFPINGLSDVKNTSEDSENFPSSSKKS...  


In [20]:
formatted_genome = [np.array(b.format_seq(seqs_genome.iloc[i].replace("*", ""))) for i in range(len(seqs_genome))]
print(len(formatted_genome))

263193


In [5]:
import pandas as pd

# loading each of the 3 HT-Recruit processed datasets 
# (tiling dataset only is shown as an example)


# nucrepr_data = pd.read_csv("data/BareNucRepr.csv", index_col=0)
# nucact_data = pd.read_csv("data/BareNucAct.csv", index_col=0)
tiles_data = pd.read_csv("AA_UniRep1900_BareTilingRepressors.csv", index_col=0)

tiles_data = tiles_data.dropna()

# print(len(nucrepr_data))
# print(len(nucact_data))
print(len(tiles_data))

print(tiles_data.columns)
# print(tiles_data[tiles_data.isnull().any(axis=1)])

# seqs_repr = nucrepr_data["Extended Domain sequence"] # should all be 80 aa's long
# seqs_act = nucact_data["Extended Domain sequence"] # should all be 80 aa's long
seqs_tiles = tiles_data["aa"] # should all be 80 aa's long

print(len(seqs_tiles))

10538
Index(['Unnamed: 0.1', 'Sequence', 'Avg D5', 'avg_UniRep_0', 'avg_UniRep_1',
       'avg_UniRep_2', 'avg_UniRep_3', 'avg_UniRep_4', 'avg_UniRep_5',
       'avg_UniRep_6',
       ...
       'avg_UniRep_1891', 'avg_UniRep_1892', 'avg_UniRep_1893',
       'avg_UniRep_1894', 'avg_UniRep_1895', 'avg_UniRep_1896',
       'avg_UniRep_1897', 'avg_UniRep_1898', 'avg_UniRep_1899', 'aa'],
      dtype='object', length=1904)
0        MTTSGALFPSLVPGSRGASNKYLVEFRAGKMSLKGTTVTPDKRKGL...
1        LVPGSRGASNKYLVEFRAGKMSLKGTTVTPDKRKGLVYIQQTDDSL...
2        KYLVEFRAGKMSLKGTTVTPDKRKGLVYIQQTDDSLIHFCWKDRTS...
3        MSLKGTTVTPDKRKGLVYIQQTDDSLIHFCWKDRTSGNVEDDLIIF...
4        DKRKGLVYIQQTDDSLIHFCWKDRTSGNVEDDLIIFPDDCEFKRVP...
5        QTDDSLIHFCWKDRTSGNVEDDLIIFPDDCEFKRVPQCPSGRVYVL...
6        WKDRTSGNVEDDLIIFPDDCEFKRVPQCPSGRVYVLKFKAGSKRLF...
7        DDLIIFPDDCEFKRVPQCPSGRVYVLKFKAGSKRLFFWMQEPKTDQ...
8        EFKRVPQCPSGRVYVLKFKAGSKRLFFWMQEPKTDQDEEHCRKVNE...
9        GRVYVLKFKAGSKRLFFWMQEPKTDQDEEHCRKVNEYL

In [10]:
# formatting sequences as integers using the UniRep formatting
# (tiles dataset is shown as an example)

# formatted_repr = [np.array(b.format_seq(seqs_repr.iloc[i])) for i in range(len(seqs_repr))]
# formatted_act = [np.array(b.format_seq(seqs_act.iloc[i])) for i in range(len(seqs_act))]
formatted_tiles = [np.array(b.format_seq(seqs_tiles.iloc[i])) for i in range(len(seqs_tiles))]
# print(len(formatted_repr))
# print(len(formatted_act))
print(len(formatted_tiles))

80
80
10538
10538


In [11]:
import unirep

# adapted from UniRep get_rep function, 1900-unit version
# returns the average hidden layer representation of each
# sequence in the formatted array of sequences.
def unirep_for_ht_recruit(f):
    b_size = len(f)
    with tf.Session() as sess:
        unirep.initialize_uninitialized(sess)

        # Final state is a cell_state, hidden_state tuple. Output is
        # all hidden states
        final_state_, hs = sess.run(
            [b._final_state, b._output], feed_dict={
                b._batch_size_placeholder: b_size,
                b._minibatch_x_placeholder: f, 
                b._initial_state_placeholder: b._zero_state}
        )

    final_cell, final_hidden = final_state_
    print(hs.shape) # is num sequences, length of sequence, length of representation

    avg_hidden = np.mean(hs, axis=1)
    return avg_hidden

# getting UniRep representations for the different HT-Recruit datasets
# (tiles dataset is shown as an example)

# avg_hidden_repr = unirep_for_ht_recruit(formatted_repr)
# avg_hidden_act = unirep_for_ht_recruit(formatted_act)
avg_hidden_tiles = unirep_for_ht_recruit(formatted_tiles)

# getting UniRep representations for the human proteome data
# has to be done in batches to avoid running out of space on the machine

# print(len(formatted_genome))
# avg_hidden_genome = []
# for i in range(21):
#     print("indices:", str(i*batch_size), str((i+1)*batch_size))
#     avg_hidden_genome.extend(unirep_for_ht_recruit(formatted_genome[int(i*batch_size): int((i+1)*batch_size)]))

(10538, 81, 1900)


In [21]:
# merging each dataset's UniRep representations with sequence data
# and writing to a separate file on the VM
# (tiling dataset is shown as an example)

# nucrepr_data = nucrepr_data.merge(pd.DataFrame(avg_hidden_repr, 
#                                 columns=["avg_UniRep_" + str(i) for i in range(avg_hidden_repr.shape[1])],
#                                 index=nucrepr_data.index,
#                                 dtype=float),
#                                  left_index=True, right_index=True)

# nucact_data = nucact_data.merge(pd.DataFrame(avg_hidden_act, 
#                                 columns=["avg_UniRep_" + str(i) for i in range(avg_hidden_act.shape[1])],
#                                 index=nucact_data.index,
#                                 dtype=float),
#                                  left_index=True, right_index=True)

tiles_data = tiles_data[["Sequence", "Avg D5", "aa"]]
tiles_data = tiles_data.merge(pd.DataFrame(avg_hidden_tiles, 
                                columns=["avg_UniRep_" + str(i) for i in range(avg_hidden_tiles.shape[1])],
                                index=tiles_data.index,
                                dtype=float),
                                left_index=True, right_index=True)

# avg_hidden_genome = np.array(avg_hidden_genome)
# genome_tiles = genome_tiles.merge(pd.DataFrame(avg_hidden_genome, 
#                                 columns=["avg_UniRep_" + str(i) for i in range(avg_hidden_genome.shape[1])],
#                                 index=genome_tiles.index,
#                                 dtype=float),
#                                 left_index=True, right_index=True)

# nucrepr_data.to_csv("data/UniRep1900_BareNucRepr.csv")
# nucact_data.to_csv("data/UniRep1900_BareNucAct.csv")
tiles_data.to_csv("data/UniRep1900_BareTilingRepressors.csv")

# !head data/UniRep1900_BareNucRepr.csv
# !head data/BareNucRepr.csv
# !head data/UniRep1900_BareNucAct.csv
!head data/UniRep1900_BareTilingRepressors.csv

Index(['Sequence', 'Avg D5', 'aa'], dtype='object')
Index(['Sequence', 'Avg D5', 'aa', 'avg_UniRep_0', 'avg_UniRep_1',
       'avg_UniRep_2', 'avg_UniRep_3', 'avg_UniRep_4', 'avg_UniRep_5',
       'avg_UniRep_6',
       ...
       'avg_UniRep_1890', 'avg_UniRep_1891', 'avg_UniRep_1892',
       'avg_UniRep_1893', 'avg_UniRep_1894', 'avg_UniRep_1895',
       'avg_UniRep_1896', 'avg_UniRep_1897', 'avg_UniRep_1898',
       'avg_UniRep_1899'],
      dtype='object', length=1903)
,Sequence,Avg D5,aa,avg_UniRep_0,avg_UniRep_1,avg_UniRep_2,avg_UniRep_3,avg_UniRep_4,avg_UniRep_5,avg_UniRep_6,avg_UniRep_7,avg_UniRep_8,avg_UniRep_9,avg_UniRep_10,avg_UniRep_11,avg_UniRep_12,avg_UniRep_13,avg_UniRep_14,avg_UniRep_15,avg_UniRep_16,avg_UniRep_17,avg_UniRep_18,avg_UniRep_19,avg_UniRep_20,avg_UniRep_21,avg_UniRep_22,avg_UniRep_23,avg_UniRep_24,avg_UniRep_25,avg_UniRep_26,avg_UniRep_27,avg_UniRep_28,avg_UniRep_29,avg_UniRep_30,avg_UniRep_31,avg_UniRep_32,avg_UniRep_33,avg_UniRep_34,avg_UniRep_35,avg_UniR

8,GAGTTCAAGAGAGTGCCCCAGTGCCCCAGCGGCAGGGTGTACGTGCTGAAGTTCAAGGCCGGCAGCAAGAGACTGTTCTTCTGGATGCAGGAGCCCAAGACCGACCAGGACGAGGAGCACTGCAGAAAGGTGAACGAGTACCTGAACAACCCCCCCATGCCTGGCGCCCTGGGCGCCAGCGGAAGCTCTGGCCATGAACTGAGCGCCCTGGGCGGCGAGGGCGGCCTGCAGAGCCTGCTG,0.1303,EFKRVPQCPSGRVYVLKFKAGSKRLFFWMQEPKTDQDEEHCRKVNEYLNNPPMPGALGASGSSGHELSALGGEGGLQSLL,0.011309478431940079,-0.07394461333751678,0.01924162544310093,0.0011523740831762552,-0.09728973358869553,0.034548453986644745,-0.2854877710342407,-0.010286235250532627,-0.012618500739336014,0.09490460157394409,0.08685141056776047,0.004956705030053854,0.04614517465233803,0.06655607372522354,0.011086476035416126,0.006037957035005093,0.01985270343720913,-0.02677701786160469,0.1720496267080307,-0.01934262178838253,-0.0019444182980805635,0.027248891070485115,0.03462718799710274,-0.07158943265676498,0.022557904943823814,-0.12485334277153015,0.05766741558909416,0.016919545829296112,0.009043872356414795,-0.07544460892677307,-0.20625022053718567,-0.08405924588441849,0.0

In [38]:
# writing human proteome UniRep representations in batches to 21 files
# for i in range(21):
#     genome_tiles.iloc[int(i*batch_size):int((i+1)*batch_size)].to_csv("/data/UniRep1900_TiledProteome_" + str(i) + ".csv")

,id,sequence,avg_UniRep_0,avg_UniRep_1,avg_UniRep_2,avg_UniRep_3,avg_UniRep_4,avg_UniRep_5,avg_UniRep_6,avg_UniRep_7,avg_UniRep_8,avg_UniRep_9,avg_UniRep_10,avg_UniRep_11,avg_UniRep_12,avg_UniRep_13,avg_UniRep_14,avg_UniRep_15,avg_UniRep_16,avg_UniRep_17,avg_UniRep_18,avg_UniRep_19,avg_UniRep_20,avg_UniRep_21,avg_UniRep_22,avg_UniRep_23,avg_UniRep_24,avg_UniRep_25,avg_UniRep_26,avg_UniRep_27,avg_UniRep_28,avg_UniRep_29,avg_UniRep_30,avg_UniRep_31,avg_UniRep_32,avg_UniRep_33,avg_UniRep_34,avg_UniRep_35,avg_UniRep_36,avg_UniRep_37,avg_UniRep_38,avg_UniRep_39,avg_UniRep_40,avg_UniRep_41,avg_UniRep_42,avg_UniRep_43,avg_UniRep_44,avg_UniRep_45,avg_UniRep_46,avg_UniRep_47,avg_UniRep_48,avg_UniRep_49,avg_UniRep_50,avg_UniRep_51,avg_UniRep_52,avg_UniRep_53,avg_UniRep_54,avg_UniRep_55,avg_UniRep_56,avg_UniRep_57,avg_UniRep_58,avg_UniRep_59,avg_UniRep_60,avg_UniRep_61,avg_UniRep_62,avg_UniRep_63,avg_UniRep_64,avg_UniRep_65,avg_UniRep_66,avg_UniRep_67,avg_UniRep_68,avg_UniRep_69,avg_UniRep_70,avg

8,ENST00000367772.8_200_280,LLTILNEQVSADVLSSFQQTLHSTLLNPIPKCRPALCTLLSHDFFRNDFLEVVNFLKSLTLKSEEEKTEFFKFLLDRVSC,0.009983714669942856,-0.034866854548454285,0.027490103617310524,0.0026024123653769493,0.12332167476415634,0.002117075026035309,-0.19956645369529724,-0.014072928577661514,-0.013437450863420963,0.015124762430787086,0.019006894901394844,0.007697838358581066,0.03939549997448921,0.11631780862808228,0.017013244330883026,0.006117536220699549,0.031171150505542755,-0.06172589212656021,0.047604382038116455,-0.009984392672777176,-7.779070438118652e-05,0.0430021733045578,0.06756163388490677,-0.10073164105415344,0.02258870005607605,0.059188783168792725,0.045213546603918076,0.013334312476217747,0.03023347444832325,-0.08474878966808319,-0.18312419950962067,-0.006305178627371788,0.013285018503665924,0.17897552251815796,-0.033757928758859634,-0.006687856744974852,0.04581879824399948,-0.029971761628985405,0.018741091713309288,0.013977357186377048,-0.01124650053679943,-0.014058645814657211,-0.0455

head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for reading: No such file or directory
head: cannot open '/data/UniRep1900_TiledProteome.csv' for r